In [1]:
import pandas as pd
df = pd.read_csv("data/Train_Test_IoT_Fridge.csv")
df.columns
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39944 entries, 0 to 39943
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                39944 non-null  object 
 1   time                39944 non-null  object 
 2   fridge_temperature  39944 non-null  float64
 3   temp_condition      39944 non-null  object 
 4   label               39944 non-null  int64  
 5   type                39944 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.8+ MB


,date,time,fridge_temperature,temp_condition,label,type
0,25-Apr-19,19:19:40,9.00,high,1,ddos
1,25-Apr-19,19:19:40,9.25,high,1,ddos
2,25-Apr-19,19:19:45,12.65,high,1,ddos
3,25-Apr-19,19:19:45,4.65,low,1,ddos
4,25-Apr-19,19:19:55,12.65,high,1,ddos


In [2]:
print(df.columns.tolist())

['date', 'time', 'fridge_temperature', 'temp_condition', 'label', 'type']


In [3]:
df['temp_condition'] = df['temp_condition'].str.strip().str.lower()
df.head()

,date,time,fridge_temperature,temp_condition,label,type
0,25-Apr-19,19:19:40,9.00,high,1,ddos
1,25-Apr-19,19:19:40,9.25,high,1,ddos
2,25-Apr-19,19:19:45,12.65,high,1,ddos
3,25-Apr-19,19:19:45,4.65,low,1,ddos
4,25-Apr-19,19:19:55,12.65,high,1,ddos


In [4]:
df = df.drop(['date','time','type'],axis=1)
df.head()

,fridge_temperature,temp_condition,label
0,9.00,high,1
1,9.25,high,1
2,12.65,high,1
3,4.65,low,1
4,12.65,high,1


In [5]:
df = pd.get_dummies(df,columns=['temp_condition'],drop_first=True)
df.head()

,fridge_temperature,label,temp_condition_low
0,9.00,1,False
1,9.25,1,False
2,12.65,1,False
3,4.65,1,True
4,12.65,1,False


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1) جدا کردن ویژگی‌ها و برچسب
X = df[['fridge_temperature', 'temp_condition_low']].copy()
X['temp_condition_low'] = X['temp_condition_low'].astype(int) # تبدیل True/False به 0/1
y = df['label']

# 2) نرمال‌سازی ویژگی‌ها
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3) تقسیم به train و test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# 4) ساخت و آموزش مدل IDS
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 5) پیش‌بینی و ارزیابی
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6262360746025786
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2986
           1       0.63      1.00      0.77      5003

    accuracy                           0.63      7989
   macro avg       0.31      0.50      0.39      7989
weighted avg       0.39      0.63      0.48      7989



C:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [7]:
y.value_counts()

label
1    24944
0    15000
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1) ویژگی‌ها و لیبل
X = df[['fridge_temperature', 'temp_condition_low']].copy()
X['temp_condition_low'] = X['temp_condition_low'].astype(int)
y = df['label']

# 2) نرمال‌سازی
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3) train / test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# 4) مدل با وزن‌دهی متعادل
clf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'
)
clf.fit(X_train, y_train)

# 5) ارزیابی
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.5016898235073226
              precision    recall  f1-score   support

           0       0.38      0.52      0.44      3000
           1       0.63      0.49      0.55      4989

    accuracy                           0.50      7989
   macro avg       0.50      0.50      0.50      7989
weighted avg       0.54      0.50      0.51      7989



In [9]:
import numpy as np

# تعداد کلاینت‌ها
num_clients = 3 
client_data = []

X = df.drop('label', axis=1).copy()
y = df['label'].copy()

# تبدیل ویژگی‌های بولین به عدد
for col in X.columns:
    if X[col].dtype == bool:
        X[col] = X[col].astype(int)

X = X.values
y = y.values

# محاسبه اندازه هر قسمت
size = len(X) // num_clients

# تقسیم دیتا
for i in range(num_clients):
    start = i * size
    end = (i + 1) * size
    client_data.append((X[start:end], y[start:end]))

# نمایش تعداد نمونه‌های هر کلاینت
for idx, (Xi, yi) in enumerate(client_data):
    print(f"Client {idx+1}: {len(Xi)} samples")


Client 1: 13314 samples
Client 2: 13314 samples
Client 3: 13314 samples


In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import SGDClassifier

df = pd.read_csv("data/Train_Test_IoT_Fridge.csv")

# پاکسازی
df["temp_condition"] = df["temp_condition"].astype(str).str.strip().str.lower()

# حذف ستون‌های اضافی مثل کاری که خودت کردی
df = df.drop(columns=["date", "time", "type"], errors="ignore")

# One-hot
df = pd.get_dummies(df, columns=["temp_condition"], drop_first=True)

# bool -> int
for col in df.columns:
    if df[col].dtype == bool:
        df[col] = df[col].astype(int)

X = df.drop(columns=["label"]).values.astype(np.float64)
y = df["label"].values.astype(int)

# train/test علمی
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Train:", X_train.shape, "Test:", X_test.shape)

Train: (31955, 2) Test: (7989, 2)


In [11]:
num_clients = 5

def split_clients(X, y, num_clients=5, non_iid=True):
    idx = np.arange(len(X))
    if non_iid:
        # Non-IID ساده: مرتب‌سازی بر اساس لیبل (مثل سناریوی بدبینانه)
        idx = idx[np.argsort(y)]
    else:
        rng = np.random.default_rng(42)
        rng.shuffle(idx)

    Xs, ys = X[idx], y[idx]
    size = len(Xs) // num_clients

    client_data = []
    for i in range(num_clients):
        start = i * size
        end = (i + 1) * size if i < num_clients - 1 else len(Xs)
        client_data.append((Xs[start:end], ys[start:end]))
    return client_data

client_data = split_clients(X_train, y_train, num_clients=num_clients, non_iid=True)

for i, (Xi, yi) in enumerate(client_data, 1):
    u, c = np.unique(yi, return_counts=True)
    print(f"Client {i}: {len(yi)} samples | dist={dict(zip(u,c))}")


Client 1: 6391 samples | dist={np.int64(0): np.int64(6391)}
Client 2: 6391 samples | dist={np.int64(0): np.int64(5609), np.int64(1): np.int64(782)}
Client 3: 6391 samples | dist={np.int64(1): np.int64(6391)}
Client 4: 6391 samples | dist={np.int64(1): np.int64(6391)}
Client 5: 6391 samples | dist={np.int64(1): np.int64(6391)}


In [12]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.utils.class_weight import compute_sample_weight

def init_model(n_features, seed=42):
    return SGDClassifier(
        loss="log_loss",
        alpha=1e-4,
        learning_rate="optimal",
        max_iter=1,
        tol=None,
        random_state=seed
    )

def local_train(Xi, yi, w_global, b_global, epochs=3):
    n_features = Xi.shape[1]
    clf = init_model(n_features)

    # وزن‌دهی به نمونه‌ها برای رفع عدم‌توازن کلاس‌ها
    sw = compute_sample_weight(class_weight="balanced", y=yi)

    # ساخت اولیه (برای اینکه coef_ و intercept_ ساخته شوند)
    clf.partial_fit(
        Xi[:10], yi[:10],
        classes=np.array([0, 1]),
        sample_weight=sw[:10]
    )

    # ست کردن وزن‌های global
    clf.coef_ = w_global.reshape(1, -1).copy()
    clf.intercept_ = np.array([b_global], dtype=np.float64)

    # آموزش محلی
    for _ in range(epochs):
        clf.partial_fit(Xi, yi, sample_weight=sw)

    w_local = clf.coef_.reshape(-1).copy()
    b_local = float(clf.intercept_[0])
    return w_local, b_local

In [13]:
def fedavg(models, sizes):
    total = float(np.sum(sizes))
    w = np.zeros_like(models[0][0], dtype=np.float64)
    b = 0.0
    for (wi, bi), ni in zip(models, sizes):
        w += (ni/total) * wi
        b += (ni/total) * bi
    return w, b

def evaluate(w, b, X_test, y_test):
    logits = X_test @ w + b
    probs = 1 / (1 + np.exp(-logits))
    y_pred = (probs >= 0.5).astype(int)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division=0))
    print("Confusion:\n", confusion_matrix(y_test, y_pred))


In [14]:
import numpy as np
print(np.unique(y_train,return_counts=True))

(array([0, 1]), array([12000, 19955]))


In [15]:

print("X_train dtype:", X_train.dtype)
print("X_test dtype:", X_test.dtype)
print("client 1 X1 dtypr:" ,client_data[0][0].dtype)
print("client 1 Y1 dtypr:" ,client_data[0][1].dtype)
print("W_global dtype:", w_global.dtype)

X_train dtype: float64
X_test dtype: float64
client 1 X1 dtypr: float64
client 1 Y1 dtypr: int64


NameError: name 'w_global' is not defined

In [ ]:

rounds = 5
local_epochs = 3
n_clusters = 2 # برای 5 کلاینت معمولاً 2 یا 3 خوبه

n_features = X_train.shape[1]
w_global = np.zeros(n_features, dtype=np.float64)
b_global = 0.0

for r in range(1, rounds+1):
    local_models = []
    sizes = []
    updates = []

    # 1) train محلی
    for (Xi, yi) in client_data:
        wi, bi = local_train(Xi, yi, w_global, b_global, epochs=local_epochs)
        local_models.append((wi, bi))
        sizes.append(len(yi))
        updates.append(np.concatenate([wi - w_global, np.array([bi - b_global])]))

    updates = np.vstack(updates)

    # 2) clustering روی آپدیت‌ها (شباهت cosine)
    sim = cosine_similarity(updates)
    dist = 1 - sim

    clustering = AgglomerativeClustering(n_clusters=n_clusters, metric="precomputed", linkage="average")
    labels = clustering.fit_predict(dist)

    print(f"\n=== Round {r} ===")
    print("Cluster labels:", labels)

    # 3) تجمیع داخل خوشه‌ها
    cluster_models = []
    cluster_sizes = []

    for c in range(n_clusters):
        members = np.where(labels == c)[0]
        models_c = [local_models[m] for m in members]
        sizes_c = [sizes[m] for m in members]

        w_c, b_c = fedavg(models_c, sizes_c)
        cluster_models.append((w_c, b_c))
        cluster_sizes.append(np.sum(sizes_c))

        print(f"Cluster {c}: clients={[int(m)+1 for m in members]} | samples={int(np.sum(sizes_c))}")

    # 4) ساخت global از خوشه‌ها
    w_global, b_global = fedavg(cluster_models, cluster_sizes)

    # 5) ارزیابی علمی روی test
    print("Global evaluation:")
    evaluate(w_global, b_global, X_test, y_test)